In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dateutil
from sklearn import ensemble,metrics,linear_model,tree
from tqdm import tqdm

# Read All data

In [2]:
campData=pd.read_csv('campaign_data.csv')
coupon_item_mappingData=pd.read_csv('coupon_item_mapping.csv')
customer_demographicsData=pd.read_csv('customer_demographics.csv')
c_trans_Data=pd.read_csv('customer_transaction_data.csv')
item_dataData=pd.read_csv('item_data.csv')
trainData=pd.read_csv('train.csv')
test_QyjYwdjData=pd.read_csv('test_QyjYwdj.csv')

In [3]:
campData['start_date']=campData['start_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['end_date']=campData['end_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['duration']=campData['end_date']-campData['start_date']
campData['duration']=campData['duration'].apply(lambda x: x.days)
campData['start_Month']=campData['start_date'].apply(lambda x: x.month)
campData['end_Month']=campData['end_date'].apply(lambda x: x.month)

In [4]:
c_trans_Data['date']=c_trans_Data['date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))

In [5]:
for j,k,l in tqdm(zip(campData['start_date'],campData['end_date'],campData['campaign_id'])):
    c_trans_Data['campaign_'+str(l).zfill(2)]=c_trans_Data['date'].apply(lambda x:1 if ((x >= j) & (x <= k)) else 0)
#     if (c_trans_Data['date'][1324561] >= j) & (c_trans_Data['date'][1324561] <= k):
#         print (j,k)

28it [01:44,  3.58s/it]


In [6]:
campCol=['campaign_24', 'campaign_25',
       'campaign_20', 'campaign_23', 'campaign_21', 'campaign_22',
       'campaign_18', 'campaign_19', 'campaign_17', 'campaign_16',
       'campaign_13', 'campaign_11', 'campaign_12', 'campaign_10',
       'campaign_09', 'campaign_08', 'campaign_07', 'campaign_06',
       'campaign_03', 'campaign_05', 'campaign_04', 'campaign_01',
       'campaign_02', 'campaign_30', 'campaign_29', 'campaign_28',
       'campaign_27', 'campaign_26']

campTran=c_trans_Data[campCol]

campTran.iloc[1324544][campTran.iloc[1324544]==1].index

campTran['campParticipated']=campTran.apply(lambda x: x[x==1].index.values, axis = 1)

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
campTran.shape

(1324566, 29)

In [11]:
transCopItemD=pd.merge(c_trans_Data,coupon_item_mappingData,on='item_id',how='left')

In [12]:
transCopItemD.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_24,campaign_25,campaign_20,...,campaign_05,campaign_04,campaign_01,campaign_02,campaign_30,campaign_29,campaign_28,campaign_27,campaign_26,coupon_id
0,2012-02-01,1501,26830,1,35.26,-10.69,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.0
1,2012-02-01,1501,26830,1,35.26,-10.69,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,20.0
2,2012-02-01,1501,26830,1,35.26,-10.69,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,29.0
3,2012-02-01,1501,54253,1,53.43,-13.89,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.0
4,2012-02-01,1501,54253,1,53.43,-13.89,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,20.0


In [13]:
transCopItemD.shape

(2657495, 36)

In [14]:
transCopItemD=transCopItemD[~transCopItemD['coupon_id'].isnull()]
transCopItemD['coupon_id']=transCopItemD['coupon_id'].map(int)
transCopItemD=transCopItemD[['date', 'customer_id', 'item_id', 'coupon_id', 'quantity', 'selling_price',
       'other_discount', 'coupon_discount']]

In [15]:
transCopItemD.head()

,date,customer_id,item_id,coupon_id,quantity,selling_price,other_discount,coupon_discount
0,2012-02-01,1501,26830,7,1,35.26,-10.69,0.0
1,2012-02-01,1501,26830,20,1,35.26,-10.69,0.0
2,2012-02-01,1501,26830,29,1,35.26,-10.69,0.0
3,2012-02-01,1501,54253,7,1,53.43,-13.89,0.0
4,2012-02-01,1501,54253,20,1,53.43,-13.89,0.0


In [ ]:
trainData.head()

In [ ]:
sns.countplot(trainData['campaign_id'])

In [ ]:
trainData.groupby('customer_id').agg({'customer_id':'count','redemption_status':'sum'}).sort_values(by='redemption_status',ascending=False)

In [ ]:
trainData.groupby('campaign_id').agg({'campaign_id':'count','redemption_status':'sum'})

In [ ]:
trainData[trainData['customer_id']==1574]

In [ ]:
c_tranCons=c_trans_Data.groupby(['customer_id','date'],as_index=False).agg({'item_id':'count','quantity':'sum','selling_price':'sum',
                                                  'other_discount':'sum','coupon_discount':'sum'})

In [ ]:

c_tranCons['coupon_availed']=c_tranCons['coupon_discount'].apply(lambda x: 1 if x !=0 else 0)

In [ ]:
c_tranCons['date']=c_tranCons['date'].apply(lambda x: dateutil.parser.parse(x)) #[c_tranCons['customer_id']==1]

In [ ]:
pdList=[]
for cid,tDF in tqdm(c_tranCons.groupby('customer_id')):
    tDF['durationVisit']=tDF['date']-tDF['date'].shift()
    tDF['durationVisit']=tDF['durationVisit'].apply(lambda x: x.days)
    pdList.append(tDF)

In [ ]:
c_tranCons2=pd.concat(pdList)

In [ ]:
trainData['dtType']='train'
test_QyjYwdjData['dtType']='test'

allData=trainData.append(test_QyjYwdjData,sort=False)
allData.shape

In [ ]:
trainData['redemption_status'].sum()

In [ ]:
campData['start_date']=campData['start_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['end_date']=campData['end_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['duration']=campData['end_date']-campData['start_date']
campData['duration']=campData['duration'].apply(lambda x: x.days)
campData['start_Month']=campData['start_date'].apply(lambda x: x.month)
campData['end_Month']=campData['end_date'].apply(lambda x: x.month)

In [ ]:
c_trans_Data['date']=c_trans_Data['date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))

In [ ]:
print (max(c_trans_Data['date']),min(c_trans_Data['date']))
print (max(campData['start_date']),min(campData['start_date']))
print (max(campData['end_date']),min(campData['end_date']))

In [ ]:
campData.sort_values('start_date')

In [ ]:
for j,k,l in tqdm(zip(campData['start_date'],campData['end_date'],campData['campaign_id'])):
    c_trans_Data['campaign_'+str(l).zfill(2)]=c_trans_Data['date'].apply(lambda x:1 if ((x >= j) & (x <= k)) else 0)
#     if (c_trans_Data['date'][1324561] >= j) & (c_trans_Data['date'][1324561] <= k):
#         print (j,k)

In [ ]:
campCol=['campaign_24', 'campaign_25',
       'campaign_20', 'campaign_23', 'campaign_21', 'campaign_22',
       'campaign_18', 'campaign_19', 'campaign_17', 'campaign_16',
       'campaign_13', 'campaign_11', 'campaign_12', 'campaign_10',
       'campaign_09', 'campaign_08', 'campaign_07', 'campaign_06',
       'campaign_03', 'campaign_05', 'campaign_04', 'campaign_01',
       'campaign_02', 'campaign_30', 'campaign_29', 'campaign_28',
       'campaign_27', 'campaign_26']

In [ ]:
campTran=c_trans_Data[campCol]

campTran.iloc[1324544][campTran.iloc[1324544]==1].index

campTran['campParticipated']=campTran.apply(lambda x: x[x==1].index.values, axis = 1)

In [ ]:
c_trans_Data['campParticipated']=campTran['campParticipated']
c_trans_Data['filt']=c_trans_Data['campParticipated'].apply(lambda x: len(x))
for j in campCol:
    del c_trans_Data[j]

In [ ]:
c_trans_Data.shape

In [ ]:
c_trans_Data=c_trans_Data[c_trans_Data['filt']!=0]
c_trans_Data['campaign_id']=c_trans_Data['campParticipated'].apply(lambda x: int(x[0].split('_')[1]))
del c_trans_Data['filt']
del c_trans_Data['campParticipated']

In [ ]:
mergedData=pd.merge(c_trans_Data,campData,on='campaign_id',how='left')

In [ ]:
mergedData['occurenceofPurch']=mergedData['date']-mergedData['start_date']
mergedData['occurenceofPurch']=mergedData['occurenceofPurch'].apply(lambda x: x.days)

In [ ]:
mergedData.head()

In [ ]:
bohotData=pd.merge(mergedData,coupon_item_mappingData,on='item_id',how='left')
# bohotData=pd.merge(bohotData,campData,on='item_id',how='left')
bohotData.shape

## Clustering operation

In [ ]:
itenACoupData=pd.merge(coupon_item_mappingData,item_dataData,on='item_id',how='left').sort_values('coupon_id')
itemInfoFromTransaction=c_trans_Data.groupby('item_id',as_index=False).agg({'selling_price':'mean',
                                                                     'coupon_discount':{'discountCount':'count','totalDisc':'sum'},
                                                                    'other_discount':{'OTdiscountCount':'count','OTtotalDisc':'sum'}})
itemInfoFromTransaction.columns=[''.join(j) for j in itemInfoFromTransaction.columns]

In [ ]:
itenACoupData=pd.merge(coupon_item_mappingData,item_dataData,on='item_id',how='left').sort_values('coupon_id')

In [ ]:
itemInfoFromTransaction['dicountPerGiven']=abs(itemInfoFromTransaction['coupon_discounttotalDisc']/itemInfoFromTransaction['coupon_discountdiscountCount'])
itemInfoFromTransaction['dicountOthPerGiven']=abs(itemInfoFromTransaction['other_discountOTtotalDisc']/itemInfoFromTransaction['other_discountOTdiscountCount'])
itemInfoFromTransaction['dicountGivenPerc']=abs(itemInfoFromTransaction['dicountPerGiven']/itemInfoFromTransaction['selling_pricemean'])
itemInfoFromTransaction['dicountOthPerGivenPerc']=abs(itemInfoFromTransaction['dicountOthPerGiven']/itemInfoFromTransaction['selling_pricemean'])


In [ ]:
transItemData=pd.merge(c_trans_Data,item_dataData,on='item_id',how='left')
transItemData.shape

transItemData['coupon_discountAvail']=transItemData['coupon_discount'].apply(lambda x: 1 if x !=0 else 0)

In [ ]:
itemCouponDetail=transItemData.groupby('item_id',as_index=False).agg({'coupon_discountAvail':{'sumOFCoupon':'sum','totalCount':'count'}})
itemCouponDetail.columns=[''.join(jj) for jj in itemCouponDetail.columns]
itemCouponDetail['probaItem']=itemCouponDetail['coupon_discountAvailsumOFCoupon']/itemCouponDetail['coupon_discountAvailtotalCount']
# itemCouponDetail[itemCouponDetail['probaItem']>0]

In [ ]:
mergedItemCoupon=pd.merge(itenACoupData,itemInfoFromTransaction,on='item_id',how='left').fillna(0)
mergedItemCoupon=pd.merge(mergedItemCoupon,itemCouponDetail[['item_id','probaItem']],on='item_id',how='left').fillna(0)
#some  null values present

In [ ]:
datatoCluster=mergedItemCoupon.groupby('coupon_id',as_index=False).mean()[['coupon_id', 
       'selling_pricemean', 'coupon_discountdiscountCount',
       'coupon_discounttotalDisc', 'other_discountOTdiscountCount',
       'other_discountOTtotalDisc', 'dicountPerGiven', 'dicountOthPerGiven',
       'dicountGivenPerc', 'dicountOthPerGivenPerc','probaItem']]

clusVar=['coupon_id', 
       'selling_pricemean', 'coupon_discountdiscountCount',
       'coupon_discounttotalDisc', 'other_discountOTdiscountCount',
       'other_discountOTtotalDisc', 'dicountPerGiven', 'dicountOthPerGiven',
       'dicountGivenPerc', 'dicountOthPerGivenPerc','probaItem']

from sklearn import preprocessing,cluster,decomposition

preProCl=preprocessing.StandardScaler()
clusData=preProCl.fit_transform(datatoCluster[clusVar])

pcaComp=decomposition.PCA()
pcaData=pcaComp.fit_transform(clusData)
pcaComp.explained_variance_ratio_

In [ ]:
pcaDataFToGraph=pd.DataFrame(pcaData[:,:2],columns=['pca1','pca2'])
kM=cluster.KMeans()
pcaDataFToGraph['clusCat']=kM.fit_predict(clusData[:,:5])
pcaDataFToGraph.head(2)
sns.scatterplot(pcaDataFToGraph['pca1'],pcaDataFToGraph['pca2'],hue=pcaDataFToGraph['clusCat'])

In [ ]:
pcaDataTOCoup=pd.concat([datatoCluster[['coupon_id']],pcaDataFToGraph],axis=1)

In [ ]:
pcaDataTOCoup.head(2)

In [ ]:
pcaDataTOCoup.shape

## Coupon Probability

In [ ]:
coupon_categFind=mergedItemCoupon.groupby(['coupon_id','category','brand','brand_type'],as_index=False).count()[['coupon_id','category','brand','brand_type','item_id']]
couponProbItemCoup=mergedItemCoupon.groupby('coupon_id',as_index=False).agg({'probaItem':'mean'})
coupon_categFind=coupon_categFind.sort_values(['coupon_id','item_id'],ascending=False)
coupon_categFind=coupon_categFind.drop_duplicates('coupon_id')
coupon_categFind=pd.merge(coupon_categFind,couponProbItemCoup,on='coupon_id',how='left')
del coupon_categFind['item_id']

In [ ]:
coupon_categFind.head(2)

In [ ]:
def getFeatureGener(df,indCol,fCol):
    groupedMerge=df.groupby(indCol)
    c_data={}
    for c_id,tempT in  groupedMerge:
        c_data[c_id]={}
        c_data[c_id][indCol+fCol+'_max']=max(tempT[fCol])
        c_data[c_id][indCol+fCol+'_min']=min(tempT[fCol])
        c_data[c_id][indCol+fCol+'_mean']=np.mean(tempT[fCol])
        c_data[c_id][indCol+fCol+'_75perc']=np.percentile(tempT[fCol],75)
        c_data[c_id][indCol+fCol+'_25perc']=np.percentile(tempT[fCol],25)
    tempDF=pd.DataFrame(c_data).transpose()
    tempDF.columns=[indCol+fCol+'_max',indCol+fCol+'_min',indCol+fCol+'_mean',indCol+fCol+'_75perc',indCol+fCol+'_25perc']
    return tempDF,tempDF.index

In [ ]:
quantityCData,indexC=getFeatureGener(mergedData,'customer_id','quantity')
selling_priceCData,indexC=getFeatureGener(mergedData,'customer_id','selling_price')
other_discountCData,indexC=getFeatureGener(mergedData,'customer_id','other_discount')
coupon_discountCData,indexC=getFeatureGener(mergedData,'customer_id','coupon_discount')
occurenceofPurchCData,indexC=getFeatureGener(mergedData,'customer_id','occurenceofPurch')

In [ ]:
featureCustomerData=pd.concat([quantityCData,selling_priceCData,other_discountCData,coupon_discountCData,occurenceofPurchCData],axis=1)

In [ ]:
featureCustomerData=featureCustomerData.reset_index()
featureCustomerData.columns=['customer_id']+list(featureCustomerData.columns[1:])

In [ ]:
featureCustomerData.head()

In [ ]:
# quantityIData,indexI=getFeatureGener(mergedData,'item_id','quantity')
# selling_priceIData,indexI=getFeatureGener(mergedData,'item_id','selling_price')
# other_discountIData,indexI=getFeatureGener(mergedData,'item_id','other_discount')
# coupon_discountIData,indexI=getFeatureGener(mergedData,'item_id','coupon_discount')
# occurenceofPurchIData,indexI=getFeatureGener(mergedData,'item_id','occurenceofPurch')

In [ ]:
# itemFeatureData=pd.concat([quantityIData,selling_priceIData,other_discountIData,coupon_discountIData,occurenceofPurchIData],axis=1)

In [ ]:
# itemFeatureData=itemFeatureData.reset_index()
# itemFeatureData.columns=['item_id']+list(itemFeatureData.columns[1:])

In [ ]:
bohotData.head()

In [ ]:
quantityCPData,indexCP=getFeatureGener(bohotData,'coupon_id','quantity')
selling_priceCPData,indexCP=getFeatureGener(bohotData,'coupon_id','selling_price')
other_discountCPData,indexCP=getFeatureGener(bohotData,'coupon_id','other_discount')
coupon_discountCPData,indexCP=getFeatureGener(bohotData,'coupon_id','coupon_discount')
occurenceofPurchCPData,indexCP=getFeatureGener(bohotData,'coupon_id','occurenceofPurch')

In [ ]:
couponFeatureData=pd.concat([quantityCPData,selling_priceCPData,other_discountCPData,coupon_discountCPData,occurenceofPurchCPData],axis=1)

In [ ]:
couponFeatureData=couponFeatureData.reset_index()
couponFeatureData.columns=['coupon_id']+list(couponFeatureData.columns[1:])

In [ ]:
couponFeatureData.head(2)

# Merge traindata with Campdata

In [ ]:
allData.head(2)

In [ ]:
X1=pd.merge(allData,featureCustomerData,on='customer_id',how='left')
X1=pd.merge(X1,couponFeatureData,on='coupon_id',how='left')
X1=pd.merge(X1,campData,on='campaign_id',how='left')
X1=pd.merge(X1,customer_demographicsData,on='customer_id',how='left')
X1=pd.merge(X1,pcaDataTOCoup,on='coupon_id',how='left')
X1=pd.merge(X1,coupon_categFind,on='coupon_id',how='left')


In [ ]:
X1.head()

In [ ]:
X5=X1

In [ ]:
X5['marital_status']=X5['marital_status'].fillna('Other')
X5['rented']=X5['rented'].fillna(0)
X5['no_of_children']=X5['no_of_children'].fillna('0')
X5['age_range']=X5['age_range'].fillna('Other')
X5['family_size']=X5['family_size'].fillna('9999')
X5['income_bracket']=X5['income_bracket'].fillna(9999)

In [ ]:
from sklearn import preprocessing,model_selection

In [ ]:
llP=preprocessing.LabelEncoder()
# X5[jj]=llP.fit_transform(X5['campaign_type'])

In [ ]:
categoColumns=['age_range', 'marital_status', 'rented', 'family_size','category', 'brand_type',
       'no_of_children', 'income_bracket','campaign_type']+['clusCat']
X5[categoColumns].head(2)#category	brand	brand_type	

In [ ]:
X5['income_bracket']=X5['income_bracket'].map(str)
X5['no_of_children']=X5['no_of_children'].map(str)
X5['clusCat']=X5['clusCat'].map(str)
X5['rented']=X5['rented'].map(str)

In [ ]:
X5.head()

In [ ]:
pd.unique(X5['family_size'])

In [ ]:

catData=pd.get_dummies(X5[categoColumns],prefix_sep='_',)
# for jj in categoColumns:
#     print (jj)
#     llP=preprocessing.LabelEncoder()
#     X5[jj]=llP.fit_transform(X5[jj])

In [ ]:
newtData=pd.concat([X5,catData],sort=False,axis=1)
# newtData=X5

In [ ]:
newtData.columns

In [ ]:
XVar=list(newtData.columns)
XVar.remove('id')
XVar.remove('redemption_status')
XVar.remove('dtType')
XVar.remove('start_date')
XVar.remove('end_date')
YVar='redemption_status'

In [ ]:
for j in categoColumns:#['campaign_id', 'coupon_id', 'customer_id']:
    XVar.remove(j)

In [ ]:
trainPart=newtData[newtData['dtType']=='train']
testPart=newtData[newtData['dtType']=='test']
trainPart.shape,testPart.shape

In [ ]:
from sklearn import model_selection,metrics
import xgboost as xgb

In [ ]:
# 	if feature_names is not None:
# 		create_feature_map(feature_names)
# 		model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
# 		importance = model.get_fscore(fmap='xgb.fmap')
# 		importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
# 		imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
# 		imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
# 		imp_df.to_csv("imp_feat.txt", index=False)

# 	pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
# 	pred_test_y2 = model.predict(xgb.DMatrix(test_X2), ntree_limit=model.best_ntree_limit)

# 	loss = 0
# 	if test_y is not None:
# 		loss = metrics.roc_auc_score(test_y, pred_test_y)
# 		return pred_test_y, loss, pred_test_y2
# 	else:
# 		return pred_test_y, loss, pred_test_y2

## Train test Split Train, Val, Test set

In [ ]:
trainX,testHX,trainY,testHY=model_selection.train_test_split(trainPart[XVar],trainPart[YVar],test_size=.2,stratify =trainPart[YVar])
trainX.shape,testHX.shape,trainY.shape,testHY.shape

In [ ]:
trainX1,valX1,trainY1,valY1=model_selection.train_test_split(trainX,trainY,test_size=.2,stratify =trainY)
trainX1.shape,valX1.shape,trainY1.shape,valY1.shape

In [ ]:
list(trainX1.columns)

In [ ]:
# def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, feature_names=None, seed_val=0, rounds=500, dep=8, eta=0.05):
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = 'auc'
params["eta"] = .05
params["subsample"] = 0.7
params["min_child_weight"] = 1
params["colsample_bytree"] = 0.7
params["max_depth"] = 8

params["silent"] = 1
params["seed"] = 200
#params["max_delta_step"] = 2
#params["gamma"] = 0.5
num_rounds = 500

plst = list(params.items())
xgtrain = xgb.DMatrix(trainX1, label=trainY1)

# if test_y is not None:
xgtest = xgb.DMatrix(valX1, label=valY1)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=20)
# else:
#     xgtest = xgb.DMatrix(test_X)
#     model = xgb.train(plst, xgtrain, num_rounds)

In [ ]:
t1=model.predict(xgtrain)
t2=model.predict(xgtest)
xgTest2 = xgb.DMatrix(testHX)
t3=model.predict(xgTest2)

# trainScore=modelR.predict_proba(trainX)
# testScore=modelR.predict_proba(valX1)
# test2Score=modelR.predict_proba(testHX)

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY1,t1),metrics.roc_auc_score(valY1,t2),metrics.roc_auc_score(testHY,t3)

In [ ]:
plst = list(params.items())
xgtrain = xgb.DMatrix(trainPart[XVar],label=trainPart[YVar])

In [ ]:
model2 = xgb.train(plst, xgtrain, num_rounds)

In [ ]:
xgTestData = xgb.DMatrix(testPart[XVar])

In [ ]:
t3Score=model.predict(xgTestData)

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:t3Score})
firstSub2.to_csv('xgbSub7.csv',index=False)